# Social Media Insight Using Naive Bayes


Text-based datasets contain a lot of information, whether they are books, historical documents, social media, e-mail, or any of the other ways we communicate via writing. Extracting features from text-based datasets and using them for classification is a difficult problem. There are, however, some common patterns for text mining. We look at disambiguating terms in social media using the Naive Bayes algorithm, which is a powerful and surprisingly simple algorithm. Naive Bayes takes a few shortcuts to properly compute the probabilities for classification, hence the term naive in the name. It can also be extended to other types of datasets quite easily and doesn't rely on numerical features. The model is a baseline for text mining studies, as the process can work reasonably well for a variety of datasets.

We will cover the following topics in this chapter:
- Downloading data from social network APIs
- Transformers for text
- Naive Bayes classifier
- Using JSON for saving and loading datasets
- The NLTK library for extracting features from text
- The F-measure for evaluation

## Disambiguation

Text is often called an **unstructured** format. There is a lot of information there, but it is just there; no headings, no required format, loose syntax and other problems prohibit the easy extraction of information from text. The data is also highly connected, with lots of mentions and cross-references—just not in a format that allows us to easily extract it!

We can compare the information stored in a book with that stored in a large database to see the difference. In the book, there are characters, themes, places, and lots of information. However, the book needs to be read and, more importantly, interpreted to gain this information. The database sits on your server with column names and data types. All the information is there and the level of interpretation needed is quite low. Information about the data, such as its type or meaning is called **metadata**, and text lacks it. A book also contains some metadata in the form of a table of contents and index but the degree is significantly lower than that of a database. 

One of the problems is the term **disambiguation**. When a person uses the word bank, is this a financial message or an environmental message (such as river bank)? This type of disambiguation is quite easy in many circumstances for humans (although there are still troubles), but much harder for computers to do.

Here, we will look at disambiguating the use of the term Python on Twitter's stream. A message on Twitter is called a tweet and is limited to 140 characters. This means there is little room for context. There isn't much metadata available although hashtags are often used to denote the topic of the tweet.

When people talk about Python, they could be talking about the following things:
- The programming language Python
- Monty Python, the classic comedy group
- The snake Python
- A make of shoe called Python

There can be many other things called Python. The aim of our experiment is to take a tweet mentioning Python and determine whether it is talking about the programming language, based only on the content of the tweet.

## Downloading data from a social network

We are going to download a corpus of data from Twitter and use it to sort out spam from useful content. Twitter provides a robust API for collecting information from its servers and this API is free for small-scale usage. It is, however, subject to some conditions that you'll need to be aware of if you start using Twitter's data in a commercial setting.

First, you'll need to sign up for a Twitter account (which is free). Go to http://twitter.com and register an account if you do not already have one.

Next, you'll need to ensure that you only make a certain number of requests per minute. This limit is currently 180 requests per hour. It can be tricky ensuring that you don't breach this limit, so it is highly recommended that you use a library to talk to Twitter's API.

You will need a key to access Twitter's data. Go to http://twitter.com and sign in to your account.

When you are logged in, go to https://apps.twitter.com/ and click on Create New App.

Create a name and description for your app, along with a website address. If you don't have a website to use, insert a placeholder. Leave the Callback URL field blank for this app—we won't need it. Agree to the terms of use (if you do) and click on Create your Twitter application.

Keep the resulting website open—you'll need the access keys that are on this page. Next, we need a library to talk to Twitter. There are many options; the one I like is simply called twitter, and is the official Twitter Python library.

Note: You can install twitter using pip3 install twitter if you are using pip to install your packages. If you are using another system,check the documentation at https://github.com/sixohsix/ twitter.

In [1]:
# import twitter
# import os
# import json

# consumer_key = "<Your Consumer Key Here>"
# consumer_secret = "<Your Consumer Secret Here>"
# access_token = "<Your Access Token Here>"
# access_token_secret = "<Your Access Token Secret Here>"
# authorization = twitter.OAuth(access_token, access_token_secret,
# consumer_key, consumer_secret)

# output_filename = os.path.join(os.path.expanduser("~"),
#  "Data", "twitter", "python_tweets.json")


# t = twitter.Twitter(auth=authorization)

# with open(output_filename, 'a') as output_file:
#     search_results = t.search.tweets(q="python", count=100)['statuses']
#     for tweet in search_results:
#         if 'text' in tweet:
#             output_file.write(json.dumps(tweet))
#             output_file.write("\n\n")

In the preceding loop, we also perform a check to see whether there is text in the tweet or not. Not all of the objects returned by twitter will be actual tweets (some will be actions to delete tweets and others). The key difference is the inclusion of text as a key, which we test for.

Running this for a few minutes will result in 100 tweets being added to the
output file.

## Loading and classifying the dataset

After we have collected a set of tweets (our dataset), we need labels to perform classification. The dataset we have stored is nearly in a JSON format. JSON is a format for data that doesn't impose much structure and is directly readable in JavaScript (hence the name, JavaScript Object Notation). JSON defines basic objects such as numbers, strings, lists and dictionaries, making it a good format for storing datasets if they contain data that isn't numerical. If your dataset is fully numerical, you would save space and time using a matrix-based format like in NumPy.

To parse it, we can use the json library but we will have to first split the file by newlines to get the actual tweet objects themselves.

## Loading data without the twitterAPI
We do not need to used the twitterAPI or anything of the like. There is a saved .txt file that has 100 tweets that we will use. If you wish to use the twitterAPI feel free to do so.

In [2]:
DATA = 'data/social-media-data/'
TWITTER = DATA + 'posts.txt'

In [16]:
tweets_list = []
with open(TWITTER, "r") as file:
    content = file.read().split('\n\n')
    for i, line in enumerate(content):
        try: 
            user, tweet = line.split('\n')
            tweets_list.append([user, tweet])
        except:
            print(f'no user: id = {i} \n tweet: {line} \n')
            tweets_list.append(['no user', tweet])

print(f"Loaded {len(tweets_list)} tweets")

no user: id = 29 
 tweet: Available Now At -… http://t.co/tvgQugZZT5 

no user: id = 31 
 tweet: No_not_that_one
RT @SamuelHLowe: - Excuse me, do you have snake belts?
- Not sure, but I can check in the back.
- Please do, my python's pants keep falling… 

no user: id = 33 
 tweet: .....))))))))))) 

no user: id = 51 
 tweet: halulan
@python_octopus 
乙です！！ 

Loaded 102 tweets


We are now interested in classifying whether an item is relevant to us or not (in this case, relevant means refers to the programming language Python). We will use the IPython Notebook's ability to embed HTML and talk between JavaScript and Python to create a viewer of tweets to allow us to easily and quickly classify the tweets as spam or not.

The code will present a new tweet to the user (you) and ask for a label: is it relevant
or not? It will then store the input and present the next tweet to be labeled.

In [17]:
labels = []

instructions = f"""
Instructions: Click in textbox. Enter a 1 if the tweet is relevant, enter 2 otherwise.
"""
print(instructions)
while len(labels) <= len(tweets_list):
    try:
        tweet_text = tweet_sample[len(labels)]['text']
    except:
        print(f'key value {len(labels)} DNE')
        labels.append(-1)

    tweet = f"""
Tweet: {tweet_sample[len(labels)]['text']}
    """
    print(tweet)
    a = input()
    try:
        if a == 'end':
            break
        val = int(a)
        if val in [1,2]:
            labels.append(int(a))
        else:
            print('invalid input: must be 1 or 2')
    except:
        print('invalid input: must be 1 or 2')
    

In [18]:
# a label with -1 is n/a
with open(DATA+'labels.txt', 'r') as f:
    content = f.read()
    label_vals = content.split('\n')
    for val in label_vals:
        labels.append(val)

In [19]:
n_samples = min(len(tweet_sample), len(labels))

## Creating a replicable dataset from Twitter

In data mining, there are lots of variables. These aren't just in the data mining algorithms—they also appear in the data collection, environment, and many other factors. Being able to replicate your results is important as it enables you to verify or improve upon your results.

**Note:** Getting 80 percent accuracy on one dataset with algorithm X, and 90 percent accuracy on another dataset with algorithm Y doesn't mean that Y is better. We need to be able to test on the same dataset in the same conditions to be able to properly compare.

Your labeling of tweets might be different from what is here. While there are obvious examples where a given tweet relates to the python programming language, there will always be gray areas where the labeling isn't obvious. One tough gray area was tweets in non-English languages that couldn't be read.

Due to these factors, it is difficult to replicate experiments on databases that are extracted from social media, and Twitter is no exception. Twitter explicitly disallows sharing datasets directly.

One solution to this is to share tweet IDs only, which you can share freely. In this section, we will first create a simulation of tweet ID dataset that we can theoretically be freely share.

In [21]:
dataset = []

for id, label in enumerate(labels):
    try:
        # print(f"tweet: {tweet_sample[id]['text']}")
        # print(f"label: {label} \n")
        dataset.append([id, label])
    except:
        # print(f'invalid key: {id} \n')
        continue

In [24]:
with open(DATA+'replicable.txt', 'w') as f:
    for item in dataset:
        f.write(str(item) + '\n')

Now that we have the tweet IDs and labels, we can recreate the original dataset if we wanted to but we will not do that here since the code up until now creates the dataset for you.

## Text transformers

Now that we have our dataset, how are we going to perform data mining on it?

Text-based datasets include books, essays, websites, manuscripts, programming code, and other forms of written expression. All of the algorithms we have seen so far deal with numerical or categorical features, so how do we convert our text into a format that the algorithm can deal with?

There are a number of measurements that could be taken. For instance, average word and average sentence length are used to predict the readability of a document. However, there are lots of feature types such as word occurrence which we will now investigate.

### Bag-of-words

One of the simplest but highly effective models is to simply count each word in the dataset. We create a matrix, where each row represents a document in our dataset and each column represents a word. The value of the cell is the frequency of that word in the document.

Here's an excerpt from The Lord of the Rings, J.R.R. Tolkien:

```
Three Rings for the Elven-kings under the sky,
Seven for the Dwarf-lords in halls of stone,
Nine for Mortal Men, doomed to die,
One for the Dark Lord on his dark throne
In the Land of Mordor where the Shadows lie.
One Ring to rule them all, One Ring to find them,
One Ring to bring them all and in the darkness bind them.
In the Land of Mordor where the Shadows lie.
 - J.R.R. Tolkien's epigraph to The Lord of The Rings
```

The word the appears nine times in this quote, while the words in, for, to, and one each appear four times. The word ring appears three times, as does the word of. 

We can create a dataset from this, choosing a subset of words and counting the frequency:

|Word| the| one| ring| to|
|--|--|--|--|--|
|Frequency| 9| 4| 3| 4|

We can use the counter class to do a simple count for a given string. When counting words, it is normal to convert all letters to lowercase, which we do when creating thestring. The code is as follows:

In [10]:
s = """Three Rings for the Elven-kings under the sky,
Seven for the Dwarf-lords in halls of stone,
Nine for Mortal Men, doomed to die,
One for the Dark Lord on his dark throne
In the Land of Mordor where the Shadows lie.
One Ring to rule them all, One Ring to find them,
One Ring to bring them all and in the darkness bind them.
In the Land of Mordor where the Shadows lie. """.lower()
words = s.split()
from collections import Counter
c = Counter(words)
c.most_common(5)

[('the', 9), ('for', 4), ('in', 4), ('to', 4), ('one', 4)]

Printing `c.most_common(5)` gives the list of the top five most frequently occurring words. Ties are not handled well as only five are given and a very large number of words all share a tie for fifth place.

The bag-of-words model has three major types. 
- The first is to use the raw frequencies, as shown in the preceding example. This does have a drawback when documents vary in size from fewer words to many words, as the overall values will be very different. 
- The second model is to use the **normalized frequency**, where each document's sum equals 1. This is a much better solution as the length of the document doesn't matter as much. 
- The third type is to simply use binary features—a value is 1 if the word occurs *at all* and 0 if it doesn't. We will use binary representation in this case.

Another popular (arguably more popular) method for performing normalization is called **term frequency - inverse document frequency**, or **tf-idf**. In this weighting scheme, term counts are first normalized to frequencies and then divided by the number of documents in which it appears in the corpus.

There are a number of libraries for working with text data in Python. We will use a major one, called **Natural Language ToolKit (NLTK)**. The `scikit-learn` library also has the `CountVectorizer` class that performs a similar action, and it is recommended you take a look at it. However the NLTK version has more options for word tokenization. If you are doing natural language processing in python, NLTK is a great library to use.

## N-grams
A step up from single bag-of-words features is that of **n-grams**. An n-gram is a
subsequence of n consecutive tokens.

They are counted the same way, with the n-grams forming a word that is put in the bag. The value of a cell in this dataset is the frequency that a particular n-gram appears in the given document.

**Note:** The value of n is a parameter. For English, setting it to between 2 to 5 is a good start, although some applications call for higher values

As an example, for n=3, we extract the first few n-grams in the following quote:

*Always look on the bright side of life.*

The first n-gram (of size 3) is Always look on, the second is look on the, the third is on the bright. As you can see, the n-grams overlap and cover three words.

Word n-grams have advantages over using single words. This simple concept introduces some context to word use by considering its local environment, without a large overhead of understanding the language computationally. A disadvantage of using n-grams is that the matrix becomes even sparser—word n-grams are unlikely to appear twice (especially in tweets and other short documents!).

Specially for social media and other short documents, word n-grams are unlikely to appear in too many different tweets, unless it is a retweet. However, in larger documents, word n-grams are quite effective for many applications.

Another form of n-gram for text documents is that of a **character n-gram**. Rather than using sets of words, we simply use sets of characters (although character n-grams have lots of options for how they are computed!). **This type of dataset can pick up words that are misspelled, as well as providing other benefits**.

## Other features (further reading)

There are other features that can be extracted too. These include syntactic features, such as the usage of particular words in sentences. Part-of-speech tags are also popular for data mining applications that need to understand meaning in text. Such feature types won't be covered in this book. If you are interested in learning more:
- *Python 3 Text Processing with NLTK 3 Cookbook, Jacob Perkins, Packt publication.*

## Naive Bayes

Naive Bayes is a probabilistic model that is unsurprisingly built upon a naive interpretation of Bayesian statistics. Despite the naive aspect, the method performs very well in a large number of contexts. It can be used for classification of many different feature types and formats, but we will focus on one: binary features in the bag-of-words model.

## Bayes' theorem

For most of us, when we were taught statistics, we started from a frequentist approach. In this approach, we assume the data comes from some distribution and we aim to determine what the parameters are for that distribution. However, those parameters are (perhaps incorrectly) assumed to be fixed. We use our model to describe the data, even testing to ensure the data fits our model.

Bayesian statistics instead model how people (non-statisticians) actually reason. We have some data and we use that data to update our model about how likely something is to occur. In Bayesian statistics, we use the data to describe the model rather than using a model and confirming it with data (as per the frequentist approach).

Bayes' theorem computes the value of $P(A|B)$, that is, knowing that $B$ has occurred,  what is the probability of $A$. In most cases, $B$ is an observed event such as it rained yesterday, and $A$ is a prediction it will rain today. For data mining, $B$ is usually we observed this sample and $A$ is it belongs to this class. We will see how to use Bayes' theorem for data mining in the next section. The equation for Bayes' theorem is given as follows:

$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}
$$

## A simple example

As an example, we want to determine the probability that an e-mail containing the word drugs is spam (as we believe that such a tweet may be a pharmaceutical spam). $A$, in this context, is the probability that this tweet is spam. We can compute $P(A)$, called the prior belief directly from a training dataset by computing the percentage of tweets in our dataset that are spam. If our dataset contains 30 spam messages for every 100 e-mails, $P(A) =  0.3$.

$B$, in this context, is this tweet contains the word 'drugs'. Likewise, we can compute $P(B)$ by computing the percentage of tweets in our dataset containing the word drugs. If 10 e-mails in every 100 of our training dataset contain the word drugs, $P(B) = 0.1$. Note that we don't care if the e-mail is spam or not when computing this value. 

$P(B|A)$ is the probability that an e-mail contains the word drugs if it is spam. It is also easy to compute from our training dataset. We look through our training set for spam e-mails and compute the percentage of them that contain the word drugs. Of our 30 spam e-mails, if 6 contain the word drugs, then $P(B|A) = 0.2$.

From here, we use Bayes' theorem to compute P(A|B), which is the probability that
a tweet containing the word drugs is spam. Using the previous equation, we see the
result is 0.6. This indicates that if an e-mail has the word drugs in it, there is a 60
percent chance that it is spam.

Note the empirical nature of the preceding example—we use evidence directly from our training dataset, not from some preconceived distribution. In contrast, a frequentist view of this would rely on us creating a distribution of the probability of words in tweets to compute similar equations.

## Naive Bayes algorithm

Looking back at our Bayes' theorem equation, we can use it to compute the probability that a given sample belongs to a given class. This allows the equation to be used as a classification algorithm.

With $C$ as a given class and $D$ as a sample in our dataset, we create the elements necessary for Bayes' theorem, and subsequently Naive Bayes. Naive Bayes is a classification algorithm that utilizes Bayes' theorem to compute the probability that a new data sample belongs to a particular class.

$P(C)$ is the probability of a class, which is computed from the training dataset itself (as we did with the spam example). We simply compute the percentage of samples in our training dataset that belong to the given class.

$P(D)$ is the probability of a given data sample. It can be difficult to compute this, as the sample is a complex interaction between different features, but luckily it is a constant across all classes. Therefore, we don't need to compute it at all. We will see later how to get around this issue.

$P(D|C)$ is the probability of the data point belonging to the class. This could also be difficult to compute due to the different features. However, this is where we **introduce the naive part of the Naive Bayes algorithm**. We naively assume that each feature is independent of each other. Rather than computing the full probability of $P(D|C)$, we compute the probability of each feature $D_1, D_2, D_3, …$ and so on. Then,
we multiply them together:

$$
P(D|C) = P(D_1|C)P(D_2|C)\cdots P(D_n|C)
$$

Each of these values is relatively easy to compute with binary features; we simply compute the percentage of times it is equal in our sample dataset.

In contrast, if we were to perform a non-naive Bayes version of this part, we would need to compute the correlations between different features for each class. Such computation is infeasible at best, and nearly impossible without vast amounts of data
or adequate language analysis models.

From here, the algorithm is straightforward. We compute $P(C|D)$ for each possible class, ignoring the $P(D)$ term. Then we choose the class with the highest probability. As the P(D) term is consistent across each of the classes, ignoring it has no impact on the final prediction.


### How it works

As an example, suppose we have the following (binary) feature values from a sample in our dataset: $D = [1, 0, 0, 1]$.

Our training dataset contains two classes with 75 percent of samples belonging to the class 0, and 25 percent belonging to the class 1. The likelihood of the feature values for each class are as follows:

For class 0: $[P(D_1=1|C=0),P(D_2=1|C=0),P(D_3=1|C=0),P(D_4=1|C=0)] = [0.3, 0.4, 0.4, 0.7]$

For class 1: $[P(D_1=1|C=1),P(D_2=1|C=1),P(D_3=1|C=1),P(D_4|=1C=1)] = [0.7, 0.3, 0.4, 0.9]$

These values are to be interpreted as: for feature 1, $D_1$, it is equal to 1 in 30 percent of cases for class 0, i.e. $P(D_1=1|C=0) = .3$.

We can now compute the probability that this sample should belong to the class 0:

$P(C=0) = 0.75$ which is the probability that the class is 0.

Note that $P(D)$ isn't needed for the Naive Bayes algorithm. Let's take a look at the calculation:

$P(D|C=0) = P(D_1|C=0)P(D_2|C=0)P(D_3|C=0)P(D_4|C=0) = 0.3\cdot 0.6 \cdot 0.6 \cdot 0.7= 0.0756$

**Note:** The listed probabilities are for values of 1 for each feature. Therefore, the probability of a 0 is its inverse: $P(0) = 1 – P(1)$.

Now, we can compute the probability of the data point belonging to this class. An important point to note is that we haven't computed P(D), so this isn't a real probability. However, it is good enough to compare against the same value for the probability of the class 1. Let's take a look at the calculation:

$$
P(C=0|D) = \frac{P(C=0) P(D|C=0)}{P(D)}
= \frac{0.75 \cdot 0.0756}{P(D)}
= \frac{0.0567}{P(D)}
$$

Now, we compute the same values for the class 1: $P(C=1) = 0.25$

$P(D)$ isn't needed for naive Bayes. Let's take a look at the calculation:

$P(D|C=1) = P(D_1|C=1)P(D_2|C=1)P(D_3|C=1)P(D_4|C=1)
= 0.7 \cdot 0.7 \cdot 0.6 \cdot 0.9
= 0.2646$

$$
P(C=1|D) = \frac{P(C=1) P(D|C=1)}{P(D)}
= \frac{0.25 \cdot 0.2646}{P(D)}
= \frac{0.06615}{P(D)}
$$

The data point should be classified as belonging to the class 1. You may have guessed this while going through the equations anyway; however, you may have been a bit surprised that the final decision was so close. After all, the probabilities in computing P(D|C) were much, much higher for the class 1. This is because we introduced a prior belief that most samples generally belong to the class 0.

If the classes had been equal sizes, the resulting probabilities would be much different. Try it yourself by changing both $P(C=0)$ and $P(C=1)$ to 0.5 for equal class sizes and computing the result again.

## Application

We will now create a pipeline that takes a tweet and determines whether it is relevant or not, based only on the content of that tweet. To perform the word extraction, we will be using the NLTK, a library that contains a large number of tools for performing analysis on natural language.

We are going to create a pipeline to extract the word features and classify the tweets using Naive Bayes. Our pipeline has the following steps:

1. Transform the original text documents into a dictionary of counts using NLTK's word_tokenize function.
2. Transform those dictionaries into a vector matrix using the DictVectorizer transformer in scikit-learn. This is necessary to enable the Naive Bayes classifier to read the feature values extracted in the first step.
3. Train the Naive Bayes classifier.

## Extracting word counts

We are going to use NLTK to extract our word counts. We still want to use it in a pipeline, but NLTK doesn't conform to our transformer interface. We will therefore need to create a basic transformer to do this to obtain both fit and transform methods, enabling us to use this in a pipeline.

First, set up the transformer class. We don't need to fit anything in this class, as this transformer simply extracts the words in the document. Therefore, our fit is an empty function, except that it returns self which is necessary for transformer objects. Our transform is a little more complicated. We want to extract each word from each document and record True if it was discovered. We are only using the binary features here—True if in the document, False otherwise. If we wanted to use the frequency we would set up counting dictionaries, as we have done in several of the past chapters.

In [11]:
len(tweet_sample.keys())

99

In [12]:
len(labels)

201

In [26]:
labels[100]

''

In [18]:

sample_tweets = [(labels[k], v) for k, v in tweet_sample.items() if k < 102]

IndexError: list index out of range

In [ ]:
import numpy as np
y_true = np.array([labels])

In [ ]:
print("{:.1f}% have class 1".format(np.mean(y_true == 1) * 100))

In [ ]:
from sklearn.base import TransformerMixin
from nltk import word_tokenize

class NLTKBOW(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [{word: True for word in word_tokenize(document)}
                 for document in X]

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:

pipeline = Pipeline([('bag-of-words', NLTKBOW()),
                     ('vectorizer', DictVectorizer()),
                     ('naive-bayes', BernoulliNB())
                     ])
scores = cross_val_score(pipeline, sample_tweets, y_true, cv=10, scoring='f1')
print("Score: {:.3f}".format(np.mean(scores)))